# Experiments for image encoding evaluation

## Setup

### Google Drive setup

In [4]:
DRIVE_MOUNT_PATH = "/content/drive"
DRIVE_WORK_DIR = "MyDrive/Colab Notebooks/GEQIE"

try:
    from google.colab import drive
    print("Environment is Google Colab.")
    WORK_DIR = f"{DRIVE_MOUNT_PATH}/{DRIVE_WORK_DIR}"

    print(f"Mounting Google Drive '{WORK_DIR}'...")
    drive.mount(DRIVE_MOUNT_PATH)

    try:
        import geqie
        print("Dependencies already installed.")
    except:
        print("Installing dependencies...")
        ! uv pip install -e .
        ! uv pip install -r experiments/requirements/requirements.in
except Exception:
    print("Environment is not Google Colab... skipping.")

Environment is not Google Colab... skipping.


### Imports

In [5]:
import os
from pathlib import Path

In [6]:
from itertools import product

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageDraw
from tqdm import tqdm

import sys
from pathlib import Path

# dodaj katalog nadrzędny do sys.path
sys.path.append(str(Path.cwd().parent))

from geqie.encodings import frqi, ifrqi, mcqi, ncqi, neqr, qrci, qualpi

### Constants

In [7]:
CURRENT_PATH = Path(os.path.abspath('.'))

GRAYSCALE_PATH = CURRENT_PATH.parent / "experiments" / "generated_images" / "grayscale"
GRAYSCALE_METHODS = [
    frqi,
    neqr,
]


RGB_PATH = CURRENT_PATH.parent / "experiments" / "generated_images" / "rgb"
RGB_METHODS = [
    ifrqi,
    mcqi,
]

In [8]:
GRAYSCALE_IMAGE_PATHS = sorted(GRAYSCALE_PATH.glob("*.png"))
RGB_IMAGE_PATHS = sorted(RGB_PATH.glob("*.png"))

### Helpers

In [9]:
def plot_images_side_by_side(original, retrieved):
	image_original = Image.fromarray(original)
	draw_original = ImageDraw.Draw(image_original)
	image_retrieved = Image.fromarray(retrieved)
	draw_retrieved = ImageDraw.Draw(image_retrieved)
	display(image_original.resize((image_original.width*30, image_original.height*30), resample=Image.NEAREST))
	display(image_retrieved.resize((image_retrieved.width*30, image_retrieved.height*30), resample=Image.NEAREST))

In [21]:
def PSNR(original, retrieved):
	mse = np.mean((original - retrieved) ** 2)
	if mse == 0:
		return float('inf')
	max_pixel = 255.0
	psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
	return psnr

## Experiment:

In [25]:
from geqie import main
from qiskit_aer.noise import NoiseModel, thermal_relaxation_error
from qiskit_aer.noise.errors import depolarizing_error
from qiskit.quantum_info import SuperOp, Kraus
import pandas as pd
from scipy.stats import pearsonr

csv_header = ['Image_name', 'PCC_clean', 'PSNR_clean', 'PCC_noised', 'PSNR_noised', 'circuit_size', 'circuit_depth']

np_results = None

shots = 1024

for (method, image_path) in tqdm(list(product(GRAYSCALE_METHODS[:-1], GRAYSCALE_IMAGE_PATHS[:2]))):
	# Create csv for results:
	method_name = getattr(method, '__name__', None).split('.')[-1]
	image_name = image_path.name.split('.')[0]
	csv_path = os.path.join(os.getcwd(), 'results')
	os.makedirs(csv_path, exist_ok=True)
	csv_path_filename = os.path.join(csv_path, method_name + '.csv')

	image = Image.open(image_path)
	image = ImageOps.grayscale(image)
	image = np.asarray(image)

	circuit = main.encode(method.init_function, method.data_function, method.map_function, image)

	# Create a global noise model with depolarizing error
	noise_model = NoiseModel()
	error_prob = 0.25  # 1% error probability
	depol_error = depolarizing_error(error_prob, circuit.num_qubits)
	noise_model.add_all_qubit_quantum_error(depol_error, ['unitary'])

	result = main.simulate(circuit, shots)
	retrieved_image = frqi.retrieve_function(result)

	result_noised = main.simulate(circuit, shots, noise_model=noise_model)
	retrieved_noised_image = frqi.retrieve_function(result_noised)

	pcc_clean_image, _ = pearsonr(image.flatten(), retrieved_image.flatten())
	pcc_noised_image, _ = pearsonr(image.flatten(), retrieved_noised_image.flatten())
	psnr_clean_image = PSNR(image, retrieved_image)
	psnr_noised_image = PSNR(image, retrieved_noised_image)

	if np_results is None:
		np_results = np.array([image_name, pcc_clean_image, psnr_clean_image, pcc_noised_image, psnr_noised_image, circuit.size(), circuit.depth()])
	else:
		np_results = np.column_stack((np_results, np.array([image_name, pcc_clean_image, psnr_clean_image, pcc_noised_image, psnr_noised_image, circuit.size(), circuit.depth()])))


df_results = pd.DataFrame(columns=csv_header, data=np_results.T)
df_results.to_csv(csv_path_filename, index=False)


100%|██████████| 2/2 [00:09<00:00,  4.60s/it]
